<a href="https://colab.research.google.com/github/eckoecho/Prediction-of-Product-Sales/blob/main/Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Prediction of Product Sales
Echo Diaz

## Project Overview


## Load and Inspect Data

## Clean Data

## Exploratory Data Analysis

## Exploratory Data Analysis